In [2]:
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models

from tqdm import tqdm

In [4]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [3]:
!git clone https://github.com/Rope-player/pytorch_advanced.git

fatal: destination path 'pytorch_advanced' already exists and is not an empty directory.


In [5]:
%cd "pytorch_advanced"

/content/pytorch_advanced


In [6]:
%cd "1_image_classification"

/content/pytorch_advanced/1_image_classification


In [7]:
%ls

1-1_load_vgg.ipynb
1-1_load_vgg_on_GoogleColab.ipynb
1-3_transfer_learning.ipynb
1_3_transfer_learning_on_GoogleColab.ipynb
1-5_fine_tuning.ipynb
data/
make_folders_and_data_downloads.ipynb
utils/


# **파인튜닝**

- 파인튜닝: 출력층 등을 변경한 모델을 학습된 모델을 기반으로 구축한 후 직접 준비한 데이터로 신경망 모델의 결함 파라미터를 학습시키는 방법.

파인튜닝은 전이학습과는 다르게 출력층 및 그 인접 부분뿐만 아니라 모든 층의 파라미터를 다시 학습시킴. (그래도 입력층에 가까운 파라미터는 학습률을 적게, 출력층에 가까운 파라미터는 학습률을 크게 설정함.)

## 데이터셋과 데이터로더 작성

데이터셋 및 데이터로더 작성 방법은 전이학습과 동일하게 함.

In [8]:
# 전이학습 실습하면서 작성한 클래스
from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

# 개미와 벌의 화상 파일 경로 리스트 작성
train_list = make_datapath_list(phase = "train")
val_list = make_datapath_list(phase = "val")


# 데이터셋 작성
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = HymenopteraDataset(
    file_list=train_list, transform = ImageTransform(size, mean, std), phase='train')

val_dataset = HymenopteraDataset(
    file_list=val_list, transform = ImageTransform(size, mean, std), phase='val')

# 데이터로더 작성
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle = True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle = False)

# 사전 객체에 정리
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


## 네트워크 모델 작성

네트워크 모델도 전이학습시 것과 동일

In [9]:
# VGG16 모델의 인스턴스 생성
use_pretrained = True
net = models.vgg16(pretrained = use_pretrained)

# VGG-16의 마지막 출력층의 출력 유닛을 개미와 벌 두 개로 지정
net.classifier[6] = nn.Linear(in_features = 4096, out_features = 2)

# 훈련모드로 설정
net.train()

print('네트워크 설정 완료: 학습된 가중치를 읽어들여 훈련 모드로 설정했습니다.')

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

네트워크 설정 완료: 학습된 가중치를 읽어들여 훈련 모드로 설정했습니다.


## 손실함수 정의

마찬가지로 전이학습때의 것. 크로스엔트로피 오차 함수를 사용하여 손실함수 정의.

In [10]:
criterion = nn.CrossEntropyLoss()

## 최적화 방법 설정

전이학습 때와는 다르게 모든 층의 파라미터를 학습할 수 있도록 옵티마이저를 설정.

먼저 각 층의 학습률을 바꿀수 있도록  파라미터 설정. VGG16의 전반부 `features` 모듈의 파라미터를 `update_param_names_1` 변수에, 후반부 classifier 모듈 중 초음 두개의 전결합층(Full-Connected Layer) 파라미터를 `update_param_names_2` 변수에, 교체한 마지막 전결합층 파라미터를 `update_param_names_2` 변수에 저장. (각각 다른 학습률 적용 가능.)

In [11]:
# 파인튜닝으로 학습할 파라미터를 각 변수에 저장.
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 학습시킬 층의 파라미터명 지정
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# 파라미터를 각 리스트에 저장
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1에 저장：", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2에 저장：", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3에 저장：", name)

    else:
        param.requires_grad = False   # 경사가 없으면 이곳으로 옴.
        print("경사 계산 없음. 학습하지 않았음：", name)

params_to_update_1에 저장： features.0.weight
params_to_update_1에 저장： features.0.bias
params_to_update_1에 저장： features.2.weight
params_to_update_1에 저장： features.2.bias
params_to_update_1에 저장： features.5.weight
params_to_update_1에 저장： features.5.bias
params_to_update_1에 저장： features.7.weight
params_to_update_1에 저장： features.7.bias
params_to_update_1에 저장： features.10.weight
params_to_update_1에 저장： features.10.bias
params_to_update_1에 저장： features.12.weight
params_to_update_1에 저장： features.12.bias
params_to_update_1에 저장： features.14.weight
params_to_update_1에 저장： features.14.bias
params_to_update_1에 저장： features.17.weight
params_to_update_1에 저장： features.17.bias
params_to_update_1에 저장： features.19.weight
params_to_update_1에 저장： features.19.bias
params_to_update_1에 저장： features.21.weight
params_to_update_1에 저장： features.21.bias
params_to_update_1에 저장： features.24.weight
params_to_update_1에 저장： features.24.bias
params_to_update_1에 저장： features.26.weight
params_to_update_1에 저장： features.26.bias


이어서 각 파라미터에 최적화 방법 설정. 모멘텀 SGD를 사용.

`params_to_update_1`의 학습률을 **1e-4**로, `params_to_update_2`는 **5e-4**, `params_to_update_2`은 **1e-3**.

momentum은 모두 **0.9**

In [14]:
# 최적화 방법 설정
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)

# [] 밖에 작성한 파라미터는 모든 params에 동일하게 적용 됨. momentum 값은 모두 동일하기에 [] 밖에 작성.

## 학습 및 검증 실시

모델을 학습시키는 `train_model` 함수 정의.

In [12]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

  # GPU가 사용 가능한지 확인 사용가능하면 device 변수에 'gpu'가 저장되ㅗ 아니라면 'cpu'가 저장됨
  # device 변수를 통해 네트워크 모델, 모델에 입력할 데이터, 라벨 데이터를 GPU에 전송
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("사용 장치:", device)

  net.to(device)

  # 네트워크가 어느 정도 고정되면 고속화 시킴.
  torch.backends.cudnn.benchmark = True


  # 에폭 반복문
  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-------------')

    # 에폭별 훈련 및 검증 반복문
    for phase in ['train', 'val']:
      if phase == 'train':
        net.train()  # 모델을 훈련 모드로
      else:
        net.eval()   # 모델을 검증 모드로

        epoch_loss = 0.0   # 에폭 손실 합
        epoch_corrects = 0 # 에폭 정답 수

        # 학습하지 않을 때의 검증 성능을 확인하기 위해 에폭이 0일때 훈련 생략
        if (epoch == 0) and (phase == 'train'):
          continue


        # 데이터로더에서 미니 배치를 꺼내서 루프
        for inputs, labels in tqdm(dataloaders_dict[phase]):

          # GPU가 사용가능하면 GPU에 데이터 전송
          inputs = inputs.to(device)
          labels = labels.to(device)

          # 옵티마이저 초기화
          optimizer.zero_grad()

          # 순전파 계산
          with torch.set_grad_enabled(phase == 'train'):
            outputs = net(inputs)
            loss = criterion(outputs, labels)  # 손실 계산
            _, preds = torch.max(outputs, 1)   # 라벨 예측

            # 훈련 시에는 오차 역전파
            if phase == 'train':
              loss.backward()
              optimizer.step()

            # 결과 계산
            # 손실의 합계 갱신
            epoch_loss += loss.item() * inputs.size(0)  
            # 정답의 합계 갱신
            epoch_corrects += torch.sum(preds == labels.data)

        epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
        epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

In [15]:
# 학습 및 검증 시행

num_epochs=2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

사용 장치: cpu
Epoch 1/2
-------------


100%|██████████| 5/5 [01:37<00:00, 19.41s/it]


val Loss: 0.7704 Acc: 0.4444
Epoch 2/2
-------------


100%|██████████| 5/5 [01:38<00:00, 19.67s/it]

val Loss: 0.7704 Acc: 0.4444


## 네트워크 저장 및 로드

저장할 경우에는 네트워크 모델의 `net`변수를 `.state_dict()`를 활용해 파라미터를 사전형 변수로 꺼낸 후 `torch.save()`로 저장. `save_path` 변수는 저장할 경로를 지정.

In [16]:
save_path = './weights_fine_tuning.pth'
torch.save(net.state_dict(), save_path) # 저장할 파라미터, 경로

불러오기를 할 경우에는 `torch.load()`로 사전형 객체를 로드하여 `net.state_dict()`에 저장.

GPU상에 저장된 파일을 CPU에 로드할 때는 `map_location`을 사용해야 함.  

In [17]:
load_path = './weights_fine_tuning.pth'
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

# GPU에 저장된 가중치를 CPU에 로드하는 경우
load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
net.load_state_dict(load_weights)

<All keys matched successfully>

파인튜닝을 사용하여 소량의 데이터로도 높은 딥러닝 성능을 낼 수 있음.